In [16]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import string
from collections import Counter, defaultdict
import re

In [17]:
POSITION_BANK = ["President", "Chancellor", "Provost", "Director", "Dean", "Controller", "Trustee", "Member", "Regent", "Chairman", "Overseer", "Assistant", "Librarian", "Secretary", "Chaplain", "Minister", "Treasurer", "Senior Counsel", "General Counsel", "Legal Counsel", "University Counsel", "College Counsel", "Special Counsel", "Corporation Counsel", "Corporate Counsel", "Officer", "Chief", "Professor", "Commissioner", "Fellow", "Chairperson", "Manager", "Clergy", "Coordinator", "Auditor", "Governor", "Representative", "Stockbroker", "Advisor", "Commandant", "Rector", "Attorney", "Curator", "Clerk", "Department Head", "Pastor", "Head", "Comptroller", "Deputy", "Inspector General", "Instructor", "Registrar", "Ombuds", "Administrator", "Liaison", "Administrative Associate", "Webmaster", "Specialist", "University Planner", "Architect"]
#for classifying position as dean, administration
DEAN_WORDS = ["summer", "student", "faculty", "academic service", "academics", "academic program", "admissions", "admission", "enrollment", "student life", "housing", "academic support", "advising", "enrollment management", 
                       "student relations", "academic computing", "academic records", "student service", "student affairs", "student development", "registrar", "financial aid", "media service", "library service", "university librar",
                       "internation affair", "special program", "continuing education", "external relation", "development", "services"]

#for subinstituion
ADMINISTRATION_WORDS = ["academic service", "academics", "academic program", "admissions", "admission", "enrollment service", "student life", "housing", "academic support", "advising", "enrollment management", 
                       "student relations", "academic computing", "academic records", "student service", "student affairs", "student development", "registrar", "financial aid", "media service", "library service", "university librar"]

In [18]:
#define keywords
keywords = {
    "Arts and Sciences": ["arts and science", "letters and science", "arts, sciences, and letters", "arts, sciences", "arts & sciences", "letters, sciences", "arts and letters", "arts & letters"],
    "Engineering": ["engineering", "mines", "aerospace", "aeronautics", "polytechnic studies", "aviation", "applied science", "computational science"],
    
    "Computer Science": ["computer science", "computer,", "software engineering", "and computer", "computing"],
    "Data Science": ["data science", "artificial intelligence", "machine learning"],
    "Technology": ["technology"],
    "Information": ["information science", "information management", "information systems", "information stud", "library science", "and information", "library stud", "informatics", "school of information", 
                    "information and", "information school"],
    "Archictecure": ["architecture", "architectural", "urban design", "urban planning", "environmental design", "environment and design", "design, construction", "school of construction"],
    

    "Behavioral Science": ["psychology", "psychological", "human science", "human ecology", "behavioral", "human and", "childhood stud"],
    "Social Sciences": ["social science", "social and", "social ecology"],
    "Natural Sciences": ['natural science', "mathematics", "chemistry", "biology", "mathematical", "physical science", "life science", "biological science", 
                        "biochemistry", "natural and", "basic", "college of science", "school of science", ", science, and", ", science and", "physics", "college of the sciences"],
    "Environmental and Earth Science": ["environmental science", "forestry", "natural resource", "college of forest", 
                              "school of forest", "school of environment ", "environmental and", "geology", "oceanography", "earth science", "geophysics", "seismology", "volcanology", "earth and" 
                              "paleontology", "mineral science", "geoscience", "earth science", "earth studies", "atmospheric", "college of the environment", "school of ecology", "school of environment"],
    "Marine Science": ["marine science", "marine stud", "ocean science", "oceanography", "oceanographic", "ocean and"],

    
    "Business": ["business", "commerce", "mba", "college of economics", "and economics", "finance", "entrepreneurship", "school of management", "and management", "accounting", "agribusiness", "economic,", "accountancy", "schools of management", "college of management"],
    "Hospitality": ["hospitality", "hotel administration", "hotel management", "restaurant management"],
    "Education": ["college of education", "school of education", "teaching", "teach", "curriculum development", "human development", "and education", ", education and"],
    "Criminal Justice": ["criminal justice", "criminology", "criminal", "forensics"],


    "Law": ["law", "legal studies"],
    "Policy and Affairs": ["public policy", "international policy", "public affairs", "international affairs", "environmental affairs", "urban affairs", "metropolitan affairs", "international service", "leadership studies", 
                           "diplomacy", "diplomat", "international studies", "public administration", "urban policy", "global studies", "international relations", "social policy", "peace studies", "global management", "policy studies",
                           "policy, planning", "school of leadership", "policy science"],
    

    "Agriculture": ["agriculture", "agricultural", "agribusiness"],
    "Family and Consumer Science": ["family and", "home economics", "consumer science", "family science", "home", "family studies"],
    "Culinary": ["culinary", "cooking", "gastronomy"],


    "Journalism": ["journalism"],
    "Communication": ["communication"],
    "Liberal Arts": ["liberal art", "liberal studies", "interdisciplinary studies", "general studies", "liberal and", "individualized study"],
    "Humanities": ["humanities", "history", "historical", "philosophy", "literature", "political"],
    "Fine Arts": ["fine arts", "music", "theater", "creative", "theatre", "film", "performing art", "the arts", "design art", "dramatic arts", "dance",
                   "visual arts", "school of drama", "recording arts", "ceramics", "fine and applied art", "school of design", "art and design", "fine", 
                   "cinema", "art ", "art, ", "studio art", "media art", "arts and performance", "drama", "arts and media", "textiles", "applied arts"],
    "Language and Culture": ["languages", "lingusitics", "cultures", "translational studies", "intercultural studies", "ethnic studies", "hebraic studies", "cultural stud", "asian studies", "culture ", " hawaiian knowledge", 
                             "jewish", "asian and pacific studies"],


    "Nursing": ["nursing"],
    "Physical Therapy and Nutrition": ["physical therapy", "nutrition", "physiotherapy", "nutritional therapy", "rehab", "human performance"],
    "Medical": ["medical", "pharmacy", "surgeons", "school of medicine", "college of medicine", "podiatric medicine", "osteopathic medicine", "human medicine"],
    "Pharmacy": ["pharmacy", "pharmacology"],
    "Dentistry" : ["dentistry", "dental", "denistry"],
    "Optometry" : ["optometry", "optics", "optical science"],
    "Veterinary" : ['veterinary'],
    "Audiology": ["audiology"],
    "Gerontology": ["gerontology"],
    "Pathology": ["pathology"],
    "Health Science": ["health science", "public health", "health profession", "college of health", "school of health", "biomedical science", "and health", "allied health", "physical education",
                        "health and", "health studies", "genome", "occupational therapy"],


    "Religious Studies": ["religious", "theology", "divinity", "biblical", "theological", "religion", "seminary", "religous", "christ college", "talmudic", "torah", "quran", "bible"],
    "Social Work": ["social work", "social welfare", "community service", "human service", "social service", "public service", "social administration"],
    

    "Graduate": ["graduate school", "graduate studies", "graduate college", "college of graduate", "school of graduate",  "advancing", "advanced study", "graduate division", "graduate and professional studies", "doctoral", "graduate study", "graduate "],
    "Continued Studies": ["extended studies", "extended education", "continuing education", "continued education", "continuing studies", "lifelong learning", "professional programs", "extended learning", "adult learning",
                           "adult education", "online learning", "adult student", "lifelong education", "continuing and", "university extension", "distance learning", "continuous education", "extended university"],
    "Professional Studies": ["professional studies", "professional development", "professional education", "school of professions", "professional campus"],
    "Administration": ["academic service", "academics", "academic program", "admissions", "admission", "enrollment service", "student life", "housing", "academic support", "advising", "enrollment management", 
                       "student relations", "academic computing", "academic records", "student service", "student affairs", "student development", "registrar", "financial aid", "media service", "library service", 
                       "university librar", "information service", "information access", "library access", "residence", "academic affairs", "library affairs", "academic succes", "first year student",
                       "undergraduate studies", "undergraduate education"],
    "Honor College": ["honor college", "honors college", "scholar program"],

    "Satellite Campus": ["campus"],
    "Foundation": ["foundation"],
    "Boards": ["board of", "regents", "trustees", "council of"],
    "Hospital": ["hospital"],
    "University College": ["university college"],
    "Error": [],
    "Dean of the College": ["dean of the college", "dean, undergraduate college", "dean of the university"]
}

In [19]:
admin_types = ["Human Resources", "Advancement", "Finance", "Planning", "Academic Affairs", "Faculty Affairs", "External Relations", "Campus Operations", "Research", "Board VP", "Administration", "Foundation", "Generic", "Enrollment" ]

DEAN_KEYWORDS = {
    # "Head VP": ["head vice president"],
    # "SubInstitution": ["arts and science", "the college", "the arts", "interdisciplinary studies", "life sciences", "agriculture", "agricultural", "liberal arts", "engineering", "law center"],
    "Generic": [],
    "Foundation": ["foundation"],


    "Administration": ["vice president, administration", "and administration", "administrative", "audit and compliance", "legal affairs", "legal services", "institutional affairs", "university affairs", "vice president, operations", 
                       "employment service", "vice president, quality", "management service", "university compliance" "university operation", "institutional services",
                       "of the corporation", "special program", "university governance", "materials management", "executive affair"," university operation", "vice president. administration", "administration",
                       "resources"],
    "Board VP": ["of the board", "board of", "board operation"],
    "Campus Operations": ["sustainability", "facilities", "campus operations", "infrastructure", "environmental", "security", "campus police", "emergency management", "safety", 
                          "campus services", "university services", "facility operation", "university architect", "land, buildings and real estate", "auxiliary", "capital construction",
                          "property management", "plant operations", "construction", "campus residence", "physical resources", "university compliance", "physical plant", "campus involvement"],
    "Health Science": ["health science", "public health", "health profession", "college of health", "school of health", "biomedical science", "and health", "allied health", "physical education",
                        "health and", "health studies", "genome"],
    "Medical": ["medical", "pharmacy", "surgeons", "school of medicine", "college of medicine", "podiatric medicine", "osteopathic medicine"],



    "Academic Affairs": ["undergraduate education", "curriculum", "provost" "undergraduate studies", "graduate studies", "instruction", "academic program", "academic development",
                         "education affair", "undergraduate affair", "academie affairs", "educational and", "teacher education", "experiential education", "academic affair", "academic administration",
                         "undergraduate initiative", "cooperative education", "school program", "interdisciplinary education", "educational outreach", "educational,", "undergraduate studies", "educational affair",
                         "academic service", "academic afair", "educational resource", "academic enrichment", "academic resource", "university studies", "vice president, academics", "academic operation",
                         "educational development", "academic issues", "academic planning", "academic department"],
    "Faculty Affairs": ["faculty", "staff relations", "faculties"],
    "Research": ["innovation", "research", "grants", "publication", "biotechnology", "scientific", "sponsored programs"],
    

    "Student Affairs": ["student", "student life", "student service", "support", "wellbeing", "well-being", "campus life", "housing", "counseling", "student affairs", "student initiative", "student service", 
                        "student success", "student activities", "student development", "students", "university life", "student learning", "residence life", "student and community",
                        "career service", "student judicial service", "advisement", "student access", "study abroad", "dining program", "academic life", "freshmen", "freshman"],
    "Enrollment": ["enrollment", "financial aid", "retention", "registrar", "admission", "enrolment"],
    

    "Information and Technology": ["technology", "information", "librar", "data management", "computer", "computing", "network service", "internet service", "electronic marketing", "converging technologies",
                    "technical service", "vice president, systems", "enterprise system", "records", "system administration"],


    "DEI": ["diversity", "equity", "inclusion", "affirmative action", "dei", "multicultural affairs", "equal opportunity", "minority affair", "intercultural affair"],
    "External Relations": [ "external", "university relation", "college relation", "public relation", "communications", "public affair", "media", "outreach", "partnership", "global",
                            "international", "study abroad", "federal relations", "public events", "institutional relation",
                           "government", "state relations", "community engagement", "community affair", "civic engagement", "community relation", "corporate relation", 
                           "foundation relation", "legislative", "policy affairs", "defense relation", "constituent relations", "regional affair", "system relation", "regional operation",
                           "federal program", "agency relation", "regional engagement", "communication", "news services", "federal affair", "commonwealth relation", "state affair", "institute relation", "community services",
                           "intercampus", "urban affairs", "public service", "school relation", "public policy", "district of columbia affair", "d.c. affair", "vice president, engagement", "vice president, policy", "institute affair",
                           "community development"],


    "Planning": ["planning", "assessment", "evaluation", "planner", "special projects", "strategy and measurement", "institutional effectiveness", "presidential initiative", "capital improvement", "strategy and policy",
                 "university projects", "institutional analysis", "commercialization strategies", "procurement", "strategic affair", "university initiative", "program development", "strategic initiative", "strategic development",
                 "strategic and", "project", "program"],
    "Finance": ["budget", "finance", "financial", "accounting", "marketing", "endowment",  "investment", "business", "fiscal", "and resource management", "institutional resources", "treasury management", "university resources", "audit", ", resource management",
                "commerce", "asset management"],
    "Advancement": ["fundraise", "fundraising", "alumni", "donation", "parent relation", "annual giving", "special events", "principal gifts", "major gift", "campaigns", 
                    "planned giving", "leadership gifts", "gift program", "trusts and estate", "giving", "philanthropy", "constituent program", "institutional advancement", "vice president, advancement", "vice president, development",
                    "university advancement", "university development", "vice president. development", "vice president. advancement", "economic advancement", "economic development",
                    "college advancement", "institute advancement", "and development", "and advancement", "resource development", "institutional advancement", "sponsored funds", "college development", "wide advancement", "institution advancement",
                    "regional development", "development", "advancement"],
    "Human Resources": ["hr", "human resource", "personnel", "recruitment", "employee relations", "union", "collective bargaining", "labor", "human relations", "human capital", "personnel", "human and", "professional development", "human service", "hiring",
                        "workforce"],
    

    "Religion": ["ministry", "mission", "ministries", "church", "religious", "religion", "spiritual", "christian development"],
    "Satellite Campus": ["satellite campus"],
    "Athletics": ["athletic", "sports"],
    # "Agriculture": ["agriculture", "agricultural"],
    # "Continued Studies": ["extended studies", "extended education", "continuing education", "continued education", "continuing studies", "lifelong learning", "professional programs", "extended learning",
    #                        "adult education", "online learning", "adult student", "lifelong education", "continuing and", "university extension", "extended program", "distance learning", "continuous education",
    #                        "online program", "online school", "online college", "online university", "distance education", "extended service"],
    "Graduate": ["graduate program", "graduate school", "graduate and", "advanced study", "graduate education", "graduate studies", "graduate "],
    "Subinstitution Match":  ["Arts and Sciences", "Engineering", "Applied Science", "Computer Science", "Data Science", "Technology", "Information", "Architecture", "Behavioral Science", "Social Sciences", "Natural Sciences", "Environmental and Earth Science", "Marine Science", "Business", "Hospitality", "Criminal Justice", "Law", "Policy and Affairs", "Agriculture", "Family and Consumer Science", "Culinary", "Journalism", "Communication", "Liberal Arts", "Humanities", "Fine Arts", "Language and Culture", "Nursing", "Physical Therapy and Nutrition", "Pharmacy", "Dentistry", "Optometry", "Veterinary", "Audiology", "Gerontology", "Pathology", "Social Work", "Continued Studies", "Professional Studies", "Honor College", "Hospital"],
    "Other": []
}

In [20]:
def extract_deans(df):
    pos = df['Position'].fillna("").str.lower()
    mask = pos.str.contains(r'dean') & ~pos.str.contains(r'assistant to the dean')
    return df.loc[mask].copy()

#remove parentheses at end of string *used for Dean (interim) or Dean (acting)
def remove_parentheses(string):
    index = string.find('(')
    if index != -1:
        string = string[:index]
    return string.title()


def assign_position_grouping(position):
    classification = ""
    for key, value in DEAN_KEYWORDS.items():
        value_upper = [s.title() for s in value]
        keyword_appears = any(s in position for s in value_upper)
        dean_comma = position.rfind("Dean,") + 5
        if key.title() == position[dean_comma:]:
            classification += key + ", "
        elif keyword_appears:
            classification += key + ", "
    return classification.strip(", ")

def assign_position_grouping_subinst(position):
    classification = ""
    for key, value in keywords.items():
        value_upper = [s.title() for s in value]
        keyword_appears = any(s in position for s in value_upper)
        dean_comma = position.rfind("Dean,") + 5
        if key.title() == position[dean_comma:]:
            classification += key + ", "
        elif keyword_appears:
            classification += key + ", "
    return classification.strip(", ")

def assign_deans_subinstitution(deans_df, subinstitution_df):
    # start fresh
    deans_df["FixedPosition"] = ""

    # lookup: (Institution, SubInstitution) → Category
    subinst_cat = (
        subinstitution_df
        .set_index(["Institution", "SubInstitution"])["Category"]
        .to_dict()
    )

    # pre-compute helpers
    pos_title = deans_df["Position"].fillna("").astype(str).str.title()
    altered   = pos_title.apply(remove_parentheses).str.strip()
    dean_idx  = altered.str.rfind("Dean") + 4
    altered_len = altered.str.len()

    has_sub   = deans_df["SubInstitution"].notna()

    mask = (dean_idx == altered_len) & has_sub
    keys = list(zip(
        deans_df.loc[mask, "Institution"],
        deans_df.loc[mask, "SubInstitution"]
    ))
    deans_df.loc[mask, "FixedPosition"] = (
        pd.Series(keys, index=deans_df.index[mask])
          .map(subinst_cat)
    )

    mask = (dean_idx == altered_len) & ~has_sub
    deans_df.loc[mask, "FixedPosition"] = "Dean Of The College"

    mask = (dean_idx != altered_len) & has_sub
    deans_df.loc[mask, "FixedPosition"] = (
        pos_title[mask].apply(assign_position_grouping_subinst)
    )

    mask = (dean_idx != altered_len) & ~has_sub
    deans_df.loc[mask, "FixedPosition"] = (
        pos_title[mask].apply(assign_position_grouping)
    )

    return deans_df

def assign_remaining(deans_df, subinstitution_df):
    df = deans_df.copy()
    # helpers
    pos_title = df["Position"].fillna("").astype(str).str.title()
    altered   = pos_title.apply(remove_parentheses).str.strip()
    dean_idx  = altered.str.rfind("Dean") + 4
    pos_len   = pos_title.str.len()

    # 1) rows whose FixedPosition was originally blank
    mask_empty = df["FixedPosition"] == ""
    df.loc[mask_empty, "FixedPosition"] = (
        pos_title[mask_empty].apply(assign_position_grouping_subinst)
    )

    # 2) same blank rows where "Dean" ends the string → Dean of the College
    mask_end = mask_empty & (dean_idx == pos_len)
    df.loc[mask_end, "FixedPosition"] = "Dean of the College"

    # 3) any row whose Position already contains that phrase
    mask_phrase = pos_title.str.contains("Dean Of The College", na=False)
    df.loc[mask_phrase, "FixedPosition"] = "Dean of the College"

    return df


def assign_none(deans_df):
    mask = deans_df['FixedPosition'] == ''
    deans_df.loc[mask, 'FixedPosition'] = 'Administration'
    return deans_df


def assign_administration(deans_df):
    pattern = '|'.join(map(re.escape, admin_types))
    mask = deans_df['FixedPosition'].str.contains(pattern, na=False)
    deans_df.loc[mask, 'FixedPosition'] = 'Administration'
    return deans_df


def process_matches(deans_df):
    mask = deans_df['FixedPosition'].str.contains('Subinstitution Match', na=False)
    deans_df.loc[mask, 'FixedPosition'] = (
        deans_df.loc[mask, 'Position'].apply(assign_position_grouping_subinst)
    )
    return deans_df


def cleanup(deans_df):
    cleanup_list = [
        "Faculty", "Administrator", "Students", "Administrative", "Enrollment",
        "Finance", "Foundation", "Student", "Academic Affair", "Programs",
        "Alumni", "Relations", "Conservatory", "Division", "Professions",
        "Practice", "Educational", "Office", "Management"
    ]
    # build a single OR-pattern, escaping any regex metacharacters
    pattern = '|'.join(map(re.escape, cleanup_list))

    # ── pre-compute helpers ──────────────────────────────────────────────
    pos_title = deans_df['Position'].fillna('').astype(str).str.title()
    altered   = pos_title.apply(remove_parentheses).str.strip()
    dean_idx  = altered.str.rfind('Dean') + 4
    not_at_end = dean_idx != altered.str.len()

    sub_title = deans_df['SubInstitution'].fillna('').astype(str).str.title()

    # does Position or SubInstitution contain a cleanup keyword?
    pos_hit = pos_title.str.contains(pattern, na=False)
    sub_hit = sub_title.str.contains(pattern, na=False)

    # rows to re-label
    mask = (
        not_at_end &
        (pos_hit | sub_hit) &
        (deans_df['FixedPosition'] != 'Administration')
    )

    deans_df.loc[mask, 'FixedPosition'] = 'Administration'
    return deans_df
        

def expand_grouping(deans_df, subinstitution_df):
    df = deans_df.copy()
    subinst_cat = subinstitution_df.set_index(
        ['Institution', 'SubInstitution'])['Category'].to_dict()

    sub_title   = df['SubInstitution'].fillna("").astype(str).str.title()
    pos_title   = df['Position'].fillna("").astype(str).str.title()

    keys = list(zip(df['Institution'], df['SubInstitution']))
    cat_series = pd.Series(keys, index=df.index).map(subinst_cat)
    key_str = df['Institution'].fillna("").astype(str) + df['SubInstitution'].fillna("").astype(str)
    fixed = df['FixedPosition']

    mask_grad = fixed.str.contains('Graduate', na=False) & (sub_title != "")
    df.loc[mask_grad, 'FixedPosition'] = cat_series[mask_grad]

    mask_grad_append = mask_grad & cat_series.notna() & ~cat_series.str.contains('Graduate', na=False)
    df.loc[mask_grad_append, 'FixedPosition'] = df.loc[mask_grad_append, 'FixedPosition'] + ', Graduate'


    mask_sat = sub_title.str.contains('Satellite Campus', na=False)
    df.loc[mask_sat, 'FixedPosition'] = df.loc[mask_sat, 'FixedPosition'] + ', Satellite Campus'

    mask_lib = pos_title.str.contains('Librar', na=False) & \
               ~key_str.str.contains('Information', case=False, na=False)
    df.loc[mask_lib, 'FixedPosition'] = 'Administration'


    mask_cont = pos_title.str.contains('Continu', na=False)
    df.loc[mask_cont, 'FixedPosition'] = 'Continued Studies'

    return df

def clean_fixed_positions(deans_df: pd.DataFrame) -> pd.DataFrame:
    """
    Collapse any runs of consecutive commas in the FixedPosition column.
    """
    # identify rows where FixedPosition contains at least two commas in a row
    mask = deans_df['FixedPosition'].str.contains(',,', na=False)
    # perform a regex replace equivalent to remove_consecutive_commas
    deans_df.loc[mask, 'FixedPosition'] = (
        deans_df.loc[mask, 'FixedPosition']
              .str.replace(r', ,+', ',', regex=True)
    )
    return deans_df
        


def categorize_subinstitutions(df: pd.DataFrame) -> pd.DataFrame:
    """
    Apply subinstitution-based FixedPosition adjustments without iterrows.
    """
    # Title-case SubInstitution for keyword tests
    subinst = df['SubInstitution'].fillna('').str.title()
    fixed = df['FixedPosition']

    # 1. Arts → Fine Arts
    mask_arts = (
        subinst.str.contains('Arts,', na=False)
        & ~fixed.str.contains('Arts and Sciences|Liberal Arts|Fine Arts', na=False)
    )
    df.loc[mask_arts, 'FixedPosition'] = fixed[mask_arts] + ', Fine Arts'

    # 2. “And Science” → Natural Sciences
    mask_natsci = (
        subinst.str.contains('And Science', na=False)
        & ~fixed.str.contains('Arts and Sciences|Natural Sciences', na=False)
    )
    df.loc[mask_natsci, 'FixedPosition'] = fixed[mask_natsci] + ', Natural Sciences'

    # 3. Remove Liberal Arts when also Arts and Sciences
    mask_libarts = (
        fixed.str.contains('Liberal Arts', na=False)
        & fixed.str.contains('Arts and Sciences', na=False)
    )
    df.loc[mask_libarts, 'FixedPosition'] = (
        fixed[mask_libarts]
            .str.replace('Liberal Arts', '', regex=False)
            .str.strip(', ')
    )

    # 4. Software Engineering → drop “Engineering” if exactly one occurrence
    eng_count = subinst.str.count(r'\bEngineering\b')
    mask_softeng = (
        subinst.str.contains('Software Engineering', na=False)
        & fixed.str.contains('Engineering', na=False)
        & (eng_count == 1)
    )
    df.loc[mask_softeng, 'FixedPosition'] = (
        fixed[mask_softeng]
            .str.replace('Engineering', '', regex=False)
            .str.strip(', ')
    )

    # 5. Hospitality → drop “Hospital,”
    mask_hosp = (
        subinst.str.contains('Hospitality', na=False)
        & fixed.str.contains('Hospital', na=False)
    )
    df.loc[mask_hosp, 'FixedPosition'] = (
        fixed[mask_hosp]
            .str.replace('Hospital,', '', regex=False)
            .str.strip(', ')
    )

    # 6. Foundation School → drop “Foundation”
    mask_found = subinst.str.contains('Foundation School', na=False)
    df.loc[mask_found, 'FixedPosition'] = (
        fixed[mask_found]
            .str.replace('Foundation', '', regex=False)
            .str.strip(', ')
    )

    return df



def add_seniority(vp_df: pd.DataFrame) -> pd.DataFrame:
    df = vp_df.copy()

    pos_lc = df['Position'].fillna("").str.lower()
    df['Seniority'] = "Default"
    mask = pos_lc.str.contains(r"\bassistant\b")
    df.loc[mask, 'Seniority'] = "Assistant"

    mask = pos_lc.str.contains(r"\bassociate\b")
    df.loc[mask, 'Seniority'] = "Associate"

    mask = pos_lc.str.contains(r"\bsenior\b")
    df.loc[mask, 'Seniority'] = "Senior"

    mask = pos_lc.str.contains(r"\bexecutive\b")
    df.loc[mask, 'Seniority'] = "Executive"

    return df


In [21]:
data_path = r"C:\Projects\connecteddatahub\data"
years = ["1999","2000","2005","2007","2008","2009","2010","2011","2013","2018"]

grouped_validated_path = os.path.join(data_path, 'cleaned_dataframes', 'subinstitutions', 'full_validation.csv')
subinstitution_df = pd.read_csv(grouped_validated_path)

In [22]:
for year in years:
    full_path = os.path.join(
        data_path, "cleaned_dataframes", f"{year}_cleanedDataframe.csv"
    )
    full_df = pd.read_csv(full_path)

    if "Designation" not in full_df.columns: full_df["Designation"]   = ""
    if "FixedPosition" not in full_df.columns: full_df["FixedPosition"] = ""

    full_df["Position"] = full_df["Position"].fillna("")

    dean_mask = (
        full_df["Position"].str.contains(r"\bDean\b", case=False, na=False) &
        ~full_df["Position"].str.contains(r"Assistant to the Dean",
                                          case=False, na=False)
    )
    dean_df = full_df.loc[dean_mask].copy()

    dean_df = assign_deans_subinstitution(dean_df, subinstitution_df)
    dean_df = process_matches(dean_df)
    dean_df = assign_remaining(dean_df, subinstitution_df)
    dean_df = assign_none(dean_df)
    dean_df = assign_administration(dean_df)
    dean_df = cleanup(dean_df)
    dean_df = expand_grouping(dean_df, subinstitution_df)
    dean_df = categorize_subinstitutions(dean_df)
    dean_df = clean_fixed_positions(dean_df)
    dean_df = add_seniority(dean_df)

    dean_df["Designation"]  = dean_df["FixedPosition"]
    dean_df["FixedPosition"] = "Dean"
    full_df.loc[dean_mask, 'Seniority'] = dean_df['Seniority']

    full_df.loc[dean_mask, ["FixedPosition", "Designation"]] = \
        dean_df[["FixedPosition", "Designation"]]

    dean_out_path = os.path.join(
        data_path, "cleaned_dataframes", "deans",
        f"{year}_deans.csv"
    )
    full_out_path = os.path.join(
        data_path, "cleaned_dataframes",
        f"{year}_cleanedDataframe.csv"
    )

    dean_df.to_csv(dean_out_path, index=False)
    full_df.to_csv(full_out_path,  index=False)

C:\Users\tykun\AppData\Local\Temp\ipykernel_19900\1389448442.py:5: DtypeWarning: Columns (7,9,12) have mixed types. Specify dtype option on import or set low_memory=False.
  full_df = pd.read_csv(full_path)
